In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
# path = '/data/yinjyun/projects/nr-vae/experiment'

In [2]:
def get_weight_from_fname(f):
    return str(Path(f)).split('_')[-1].split('.')[0]

def get_content(x, n_met):
    bound = n_met * 7 + 2 
    models = x[:5]
    scores = x[5:5+bound]
    avg_scores = x[5+bound:]

    return models, scores, avg_scores

def process_score(x):
    return json.loads(''.join(x))

def get_weight_config(model_path):
    model_id = str(model_path).split('/')[-1]
    weight_config = '-'.join(str(Path(model_path).parent).split('/')[-1].split('-')[-7:-1])
    return weight_config
    # return '-'.join([model_id, weight_config])
    
def check_match(wc_from_fname, models_from_content):
    tmp = [get_weight_config(m) for m in models_from_content]
    print(tmp)
    for i in tmp:

        assert '-'.join(i.split('-')[-6:]) == wc_from_fname
    return tmp
        
def get_summary(path, pattern, targets, get_stat=True):
    dict_wc = {}
    model_id = []

    for i, f in enumerate(Path(path).glob('*%s*' % pattern)):
        #print(f)
        with open(f) as x:
            wc = get_weight_from_fname(f)
            # print(i, wc)
            y = [line.strip() for line in x]
            
            if pattern == 'batch_eval_sample':
                n_met = 2
            else:
                n_met = 3
            models, scores, avg_scores = get_content(y, n_met)

#             ids = check_match(wc, models)
#             model_id.extend(ids)
    
            dict_wc.update({wc: None})
            dict_summary = {k: {} for k in targets}

            scores = process_score(scores)
            avg_scores = process_score(avg_scores)

            for k in targets:
                multiplier = 1
                if k == 'acc' or k == 'f1':
                    multiplier = 100
                
                if get_stat:
                    dict_summary[k].update(
                        {#'best': max(scores[k]),
                         'std': np.std(np.array(scores[k]))*multiplier,
                          'avg': avg_scores[k]*multiplier})
                else:
                    dict_summary[k] = np.array(scores[k]) * multiplier

            dict_wc[wc]=(dict_summary)
    return model_id, dict_wc

def get_aggregated(path, append=None):
    # path = Path().resolve() / 'score' / 'ps=2-l=8'
    model_id1, dict_lc = get_summary(path, 'batch_eval_lc', targets=['pitch', 'instrument', 'combine'])
    # model_id2, dict_met = get_summary(path, 'batch_eval_met', targets=['f1', 'acc', 'nmi'])
    model_id2, dict_met = get_summary(path, 'batch_eval_met', targets=['acc', 'f1'])
    # model_id3, dict_cd = get_summary(path, 'batch_eval_cond_gen', targets=['con', 'div', 'cd'])
    model_id4, dict_rand = get_summary(path, 'batch_eval_sample', targets=['fid_recon', 'fid_rand'])
    model_id3, dict_cd = get_summary(path, 'batch_eval_cond_gen', targets=['cd'])


    for k in dict_lc.keys():
        dict_lc[k].update(dict_met[k])
        dict_lc[k].update(dict_rand[k])
        dict_lc[k].update(dict_cd[k])

    df = pd.concat({k: pd.DataFrame(v) for k, v in dict_lc.items()}, axis=0)
    if append is not None:
        return pd.concat([append, df])
    return df

In [4]:
# 7/15 t-test
import scipy.stats as ss

txt_file = ['batch_eval_lc', 'batch_eval_met', 'batch_eval_sample', 'batch_eval_cond_gen']
met = [['pitch', 'instrument', 'combine'], ['acc', 'f1'], ['fid_recon', 'fid_rand'], ['cd']]
best = [['10-1-1-1-1-1', '10-1-1-1-1-0', '10-1-1-1-1-1'], ['10-1-0-0-1-0', '10-1-0-0-1-0'], ['10-1-1-1-1-1', '10-1-1-1-1-1'], ['10-1-1-1-1-1']]
for i, j, k in zip(txt_file, met, best):
    # _, dict_base = get_summary(Path().resolve() / 'score_final/' / 'ps=0-su=1.0-l=8-10-1-0-0-0-0-bf=True', i, j, False)
    # _, dict_base = get_summary(Path().resolve() / 'score_final/' / 'ps=0-su=0.0-l=8-10-1-0-0-0-0-bf=True', i, j, False)
    _, dict_wc = get_summary(Path().resolve() / 'score_final' / 'ps=2-su=0.0-l=8', i, j, False)
    # print(i, j, dict_wc.keys())
    
    if len(j) > 1:
        for m, b in zip(j, k):
            # base_scores = np.array(dict_base[list(dict_base.keys())[0]][m])
            base_scores = np.array(dict_wc['10-1-0-0-0-0'][m])
            best_scores = np.array(dict_wc[b][m])
            # print(m, list(dict_base.keys())[0], base_scores, base_scores.mean())
            print(m, '10-1-0-0-0-0', base_scores, base_scores.mean())
            print(m, b, best_scores, best_scores.mean())
            print(ss.ttest_ind(base_scores, best_scores, equal_var=False)[1], ss.ttest_rel(base_scores, best_scores)[1])
    else:
        j = j[0]
        k = k[0]
        # base_scores = np.array(dict_base[list(dict_base.keys())[0]][j])
        base_scores = np.array(dict_wc['10-1-0-0-0-0'][j])
        best_scores = np.array(dict_wc[b][j])
        # print(j, list(dict_base.keys())[0], base_scores, base_scores.mean())
        print(m, '10-1-0-0-0-0', base_scores, base_scores.mean())
        print(j, k, best_scores, best_scores.mean())
        print(ss.ttest_ind(base_scores, best_scores, equal_var=False)[1], ss.ttest_rel(base_scores, best_scores)[1])

pitch 10-1-0-0-0-0 [24.85875706 25.42372881  6.77966102 15.25423729  9.60451977] 16.38418079096045
pitch 10-1-1-1-1-1 [12.42937853 14.12429379 18.6440678  15.25423729 12.42937853] 14.576271186440678
0.671048419404872 0.711574260022827
instrument 10-1-0-0-0-0 [85.87570621 86.44067797 83.61581921 85.87570621 90.39548023] 86.4406779661017
instrument 10-1-1-1-1-0 [87.00564972 85.87570621 87.00564972 89.83050847 89.83050847] 87.909604519774
0.3167828074091064 0.19900935971080042
combine 10-1-0-0-0-0 [80.50847458 80.50847458 88.4180791  85.31073446 90.39548023] 85.02824858757063
combine 10-1-1-1-1-1 [89.5480226  85.31073446 83.33333333 85.02824859 86.44067797] 85.9322033898305
0.7036159897499936 0.7517059942417703
acc 10-1-0-0-0-0 [75.70621469 76.83615819 88.70056497 71.75141243 79.66101695] 78.53107344632768
acc 10-1-0-0-1-0 [81.3559322  80.79096045 77.40112994 81.92090395 80.79096045] 80.45197740112995
0.5459288434175844 0.623375926575654
f1 10-1-0-0-0-0 [77.96610169 77.96610169 90.9604519

In [9]:
get_summary(Path().resolve() / 'score_final/' / 'ps=0-su=1.0-l=8-10-1-0-0-0-0-bf=True', 'batch_eval_lc', ['pitch', 'instrument', 'combine'], )

([],
 {'10-1-0-0-0-0': {'pitch': {'std': 3.4753799998259205,
    'avg': 8.8135593220339},
   'instrument': {'std': 1.0955208717325065, 'avg': 87.68361581920905},
   'combine': {'std': 1.853245137497934, 'avg': 89.43502824858759}}})

In [3]:
# score_final is the final score reported in the paper (checked 7/13)
df = None
df = get_aggregated(Path().resolve() / 'score_final/' / 'ps=0-su=1.0-l=8-10-1-0-0-0-0-bf=True')
df = get_aggregated(Path().resolve() / 'score_final' / 'ps=0-su=0.0-l=8-10-1-0-0-0-0-bf=True', df)
df = get_aggregated(Path().resolve() / 'score_final/' / 'ps=2-su=0.0-l=8', df)
# df = get_aggregated(Path().resolve() / 'score' / 'ps=2-l=8-s=1.0', df)
# df = get_aggregated(Path().resolve() / 'score' / 'ps=1-l=8', df)
# df = get_aggregated(Path().resolve() / 'score' / 'ps=3-l=8', df)

10-1-0-0-0-0
10-1-0-0-0-0
10-1-0-0-0-0
10-1-0-0-0-0
10-1-0-0-0-0
10-1-0-0-0-0
10-1-0-0-0-0
10-1-0-0-0-0
10-1-0-0-1-0
10-1-0-1-0-0
10-1-0-0-0-0
10-1-1-1-1-0
10-1-1-0-0-0
10-1-1-1-1-1
10-1-1-1-1-1
10-1-0-0-0-0
10-1-0-0-1-0
10-1-1-1-1-0
10-1-1-0-0-0
10-1-0-1-0-0
10-1-0-1-0-0
10-1-1-1-1-1
10-1-1-1-1-0
10-1-0-0-0-0
10-1-1-0-0-0
10-1-0-0-1-0
10-1-1-0-0-0
10-1-1-1-1-1
10-1-0-0-1-0
10-1-0-1-0-0
10-1-1-1-1-0
10-1-0-0-0-0


In [4]:
df

pitch  instrument    combine        acc         f1  \
10-1-0-0-0-0 avg   8.813559   87.683616  89.435028  95.141243  96.045198   
             std   3.475380    1.095521   1.853245   0.985062   0.714639   
             avg  33.785311   80.903955  73.559322  72.655367  74.463277   
             std   7.387100    4.411123   5.779857   4.828443   4.064657   
             avg  16.384181   86.440678  85.028249  78.531073  80.225989   
             std   7.655315    2.202665   4.030748   5.683515   6.011045   
10-1-0-0-1-0 avg  32.542373   84.180791  75.819209  80.451977  82.711864   
             std   6.287199    1.924224   4.080332   1.581921   1.268358   
10-1-0-1-0-0 avg  20.451977   84.745763  82.146893  77.401130  79.096045   
             std   7.985112    2.673934   5.170032   5.015218   6.084932   
10-1-1-0-0-0 avg  17.853107   87.344633  84.745763  77.288136  78.757062   
             std   4.528241    1.268358   2.539232   3.471704   3.601667   
10-1-1-1-1-0 avg  18.192090   87.909605  84.858757  78.192090  79.661017   
             std   4.799268    1.621774   2.485876   2.359391   2.813537   
10-1-1-1-1-1 avg  14.576271   86.440678  85.932203  79.887006  80.903955   
             std   2.299095    2.551772   2.063501   1.842882   2.185207   

                  fid_recon   fid_rand         cd  
10-1-0-0-0-0 avg  21.803688  23.789847  24.338662  
             std   1.052373   1.471550   0.710619  
             avg  24.860820  25.277411   8.496221  
             std   2.273966   1.806634   1.962023  
             avg  23.937316  26.405311  11.458366  
             std   1.974982   2.398934   2.341349  
10-1-0-0-1-0 avg  18.684091  20.828546  10.793080  
             std   2.367891   1.676817   2.378997  
10-1-0-1-0-0 avg  26.001726  26.907238   9.203264  
             std   1.785452   2.289646   1.550139  
10-1-1-0-0-0 avg  18.869533  21.537090   9.159448  
             std   1.774465   1.103613   1.282169  
10-1-1-1-1-0 avg  16.737443  21.398559   9.353753  
             std   2.139496   2.495492   2.815465  
10-1-1-1-1-1 avg  13.762331  19.189235  13.468186  
             std   1.074243   1.909484   1.642891

In [6]:
# 10001 is to add the result with only l4 is on, 
# and 11111 is to see if bf=False hurt which does and hence is not reported (checked 7/13)
get_aggregated(Path().resolve() / 'score_final/' / '0516_extra_exp', None)

10-1-0-0-0-1
10-1-1-1-1-1
10-1-1-1-1-1
10-1-0-0-0-1
10-1-1-1-1-1
10-1-0-0-0-1
10-1-1-1-1-1
10-1-0-0-0-1


pitch  instrument    combine        acc         f1  \
10-1-0-0-0-1 avg  17.062147   84.180791  83.559322  74.350282  75.593220   
             std   3.838483    1.383893   1.842882   2.758544   3.321342   
10-1-1-1-1-1 avg  19.435028   87.457627  84.011299  74.350282  74.576271   
             std   3.325184    1.095521   2.041730   6.860162   6.992881   

                  fid_recon   fid_rand         cd  
10-1-0-0-0-1 avg  22.361677  24.745731  11.995651  
             std   2.360308   2.174029   2.679326  
10-1-1-1-1-1 avg  13.688851  18.923793  11.273684  
             std   2.228957   1.788411   2.391493

In [ ]:
10-1-1-1-1-1
10-1-1-0-0-0
10-1-0-0-0-0

In [ ]:
path = Path().resolve() / 'score' / 'ps=2-l=8-s=1.0'
model_id1, dict_lc = get_summary(path, 'batch_eval_lc', targets=['family', 'instrument', 'pitch'])
# model_id2, dict_met = get_summary(path, 'batch_eval_met', targets=['f1', 'acc', 'nmi'])
model_id2, dict_met = get_summary(path, 'batch_eval_met', targets=['f1', 'acc'])
model_id3, dict_cd = get_summary(path, 'batch_eval_cond_gen', targets=['cd'])
model_id4, dict_rand = get_summary(path, 'batch_eval_sample', targets=['fid_recon', 'fid_rand'])

In [ ]:
for k in dict_met.keys():
    dict_met[k].update(dict_lc[k])
    dict_met[k].update(dict_cd[k])
    dict_met[k].update(dict_rand[k])

df2 = pd.concat({k: pd.DataFrame(v) for k, v in dict_met.items()}, axis=0)

In [ ]:
path = Path().resolve() / 'score' / 'ps=2-l=8-s=0.1'
model_id1, dict_lc = get_summary(path, 'batch_eval_lc', targets=['family', 'instrument', 'pitch'])
# model_id2, dict_met = get_summary(path, 'batch_eval_met', targets=['f1', 'acc', 'nmi'])
model_id2, dict_met = get_summary(path, 'batch_eval_met', targets=['f1', 'acc'])
model_id3, dict_cd = get_summary(path, 'batch_eval_cond_gen', targets=['cd'])
model_id4, dict_rand = get_summary(path, 'batch_eval_sample', targets=['fid_recon', 'fid_rand'])

In [ ]:
for k in dict_met.keys():
    dict_met[k].update(dict_lc[k])
    dict_met[k].update(dict_cd[k])
    dict_met[k].update(dict_rand[k])

df3 = pd.concat({k: pd.DataFrame(v) for k, v in dict_met.items()}, axis=0)

In [ ]:
path = Path().resolve() / 'score' / 'ps=0-l=8'
model_id1, dict_lc = get_summary(path, 'batch_eval_lc', targets=['family', 'instrument', 'pitch'])
# model_id2, dict_met = get_summary(path, 'batch_eval_met', targets=['f1', 'acc', 'nmi'])
model_id2, dict_met = get_summary(path, 'batch_eval_met', targets=['f1', 'acc'])
# model_id3, dict_cd = get_summary(path, 'batch_eval_cond_gen', targets=['con', 'div', 'cd'])
model_id3, dict_cd = get_summary(path, 'batch_eval_cond_gen', targets=['cd'])
model_id4, dict_rand = get_summary(path, 'batch_eval_sample', targets=['fid_recon', 'fid_rand'])

In [ ]:
for k in dict_met.keys():
    dict_met[k].update(dict_lc[k])
    dict_met[k].update(dict_cd[k])
    dict_met[k].update(dict_rand[k])

df0 = pd.concat({k: pd.DataFrame(v) for k, v in dict_met.items()}, axis=0)

In [ ]:
pd.concat([df0, df1, df3, df2])

In [ ]:
import numpy as np

eps = np.finfo(float).eps

def luo_met(py_c):
    assert py_c.ndim == 2
    assert np.allclose(py_c.sum(axis=1), np.ones(py_c.shape[0]))
    exp_py_c = py_c.mean(axis=0)
    h_py_c = - (py_c * np.log(py_c + eps)).sum(axis=1)
    nom = - h_py_c.mean(axis=0)
    # print(h_py_c)
    h_exp_py_c = - (exp_py_c * np.log(exp_py_c + eps)).sum()
    # print(h_exp_py_c)
    den = h_exp_py_c
    return np.exp((nom + den))



In [ ]:
x1 = np.ones([82, 82]) / 82
x2 = np.zeros([82, 82])
x2[np.arange(82), np.arange(82)] = 1
print(luo_met(x1))
print(luo_met(x2))

In [ ]:
import numpy as np

In [ ]:
x1.sum(axis=1)

In [ ]:
luo_metv2(x2)